In [11]:
import tarfile as tar
import os
import time
import numpy as np
import pandas as pd
from PIL import Image
import io
import pickle
import gzip
from keras.preprocessing.image import ImageDataGenerator

In [12]:
def imagen2array (input_loc, output_loc):
    """Funcion para convertir las imagenes que se encuentran comprimidas en archivos .tar
    a arrays normalizados.
    Argumentos:
        input_loc: directorio de los archivos .tar.
        output_loc: directorio donde se quieren almacenar los archivos pickle con los arrays normalizados.
    Devuelve:
        Un archivo csv con resumen de los archivos procesados
    """    
    ## Si el directorio de salida no esta creado, lo creo
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    
    ## Guardo los nombres de los archivos que estan en el directorio de entrada
    archivos_comprimidos = os.listdir(input_loc)
    #Solo los que tengan extencion .tar
    archivos_comprimidos = [arch for arch in archivos_comprimidos if arch.endswith('.tar')]
    
    # Tiempo de inicio global
    time_start_g = time.time()
    
    info_imagenes = pd.DataFrame()
    
    for archivo in archivos_comprimidos:
        print('Procesando el archivo: '+ archivo + '\n')
        print ('Archivo %d' %(archivos_comprimidos.index(archivo)+1) +" de un total de %d" %(len(archivos_comprimidos)))
        
        # Tiempo de inicio
        time_start = time.time()
        
        with tar.open(input_loc + archivo, 'r:gz') as tf:
            
            ##Ordeno la lista de los archivos dentro del tar
            list_arch_c = tf.getnames()
            list_arch_c = sorted(list_arch_c, key=lambda list_arch_c: int(list_arch_c[-9:-4])) #en key de cada nombre 
            
            for archivo_c in list_arch_c:
                ##Evaluo si es la primera imagen
                if list_arch_c.index(archivo_c) == 0:
                    #traigo el archivo dentro del tar
                    tarinfo = tf.getmember(archivo_c)
                    #extraigo el archivo
                    image = tf.extractfile(tarinfo)
                    #Lo leo como imagen
                    image = image.read()
                    image = Image.open(io.BytesIO(image))
                    #lo transformo a array
                    image = np.array(image)
                    #Como voy a guardar varias imagenes en un mismo array modifico el array para que sea 
                      #3D, la primer componente me indica la imagen, la segunda el ancho de la imagen y la tercera el largo
                    image = image.reshape(1,image.shape[0], image.shape[1], 1)
                    
                    ##Modifico cada imagen para que no sea igual a la anterior
                    # Create un objeto para trasnformar la imagen
                    data_generator = ImageDataGenerator(rotation_range = 5,
                                                        width_shift_range = 0.1,
                                                        height_shift_range = 0.1,
                                                        shear_range = 0.2,
                                                        zoom_range = 0.2,
                                                        horizontal_flip = True,
                                                        fill_mode = 'nearest')
                    j = 0
                    for batch in data_generator.flow(image):
                        j += 1
                        image = batch.reshape((batch.shape[0],batch.shape[1],batch.shape[2]))
                        if j % 1 == 0:  # Generate una sola imagen
                            break  #para salir del loop infinito
                            
                    #Como es la primera imagen solo creo un nuevo objeto llamado images_arrays. Este objeto almacenará todas las imagenes del tar
                    images_arrays = image
                    
                    #Creo un diccionario con los nombres de las imagenes. La posicion de las imagenes en el array
                    dic = {'Nombre_archivo_video': archivo, 
                           'Nombre_imagen': archivo_c[archivo_c.rfind('/')+1:]
                          }
                    #Convierto el diccionario en data.frame
                    data = pd.DataFrame(data=[dic], columns = ['Nombre_archivo_video', 'Nombre_imagen'])
                    #Agrego la info de los videos ya procesados
                    info_imagenes = info_imagenes.append(data)
                    
                else:
                    tarinfo = tf.getmember(archivo_c)
                    image = tf.extractfile(tarinfo)
                    image = image.read()
                    image = Image.open(io.BytesIO(image))
                    image = np.array(image)
                    image = image.reshape(1,image.shape[0], image.shape[1], 1)    
                    
                     ##Modifico cada imagen para que no sea igual a la anterior
                    # Create un objeto para trasnformar la imagen
                    data_generator = ImageDataGenerator(rotation_range = 5,
                                                        width_shift_range = 0.01,
                                                        height_shift_range = 0.01,
                                                        shear_range = 0.2,
                                                        zoom_range = 0.1,
                                                        horizontal_flip = True,
                                                        fill_mode = 'nearest')
                    j = 0
                    for batch in data_generator.flow(image):
                        j += 1
                        image = batch.reshape((batch.shape[0],batch.shape[1],batch.shape[2]))
                        if j % 1 == 0:  # Generate una sola imagen
                            break  #para salir del loop infinito
                    
                    
                    #Concateno los arreys de imagenes
                    images_arrays = np.concatenate((images_arrays, image), axis=0)
                    
                    #Creo un diccionario con los nombres de las imagenes. La posicion de las imagenes en el array
                    dic = {'Nombre_archivo_video': archivo, 
                           'Nombre_imagen': archivo_c[archivo_c.rfind('/')+1:]
                          }
                    #Convierto el diccionario en data.frame
                    data = pd.DataFrame(data=[dic], columns = ['Nombre_archivo_video', 'Nombre_imagen'])
                    #Agrego la info de los videos ya procesados
                    info_imagenes = info_imagenes.append(data)
        
        ##Normalizo los arrays dividiendo por 255 que es la escala de grisis
        images_arrays = images_arrays/255
        
        ##Guardo el array en gzip
        nombre_archivo = archivo[:-4]
        with gzip.GzipFile(output_loc+nombre_archivo+'.pgz','wb') as f:
            pickle.dump(images_arrays, f)
        
        # Tiempo final
        time_end = time.time()
        
        print('El archivo tardó en ser procesado: %d' %((time_end-time_start))+'seg')
        print("-----------------------------------------\n")
    
    # Tiempo final
    time_end_g = time.time()
    print('El proceso global tardó %d' %((time_end_g - time_start_g)/60) +'min')
    
    info_imagenes.to_csv('info_imagenes.csv', sep =';', encoding='utf-8', index=False)

In [14]:
imagen2array ('./Imagenes/Comprimidos/' , './Arrays_imagenes/')

Procesando el archivo: NoPico_Atras_I5_20190410_102959.tar

Archivo 1 de un total de 10
El archivo tardó en ser procesado: 75seg
-----------------------------------------

Procesando el archivo: NoPico_Atras_I2_20190410_113059.tar

Archivo 2 de un total de 10
El archivo tardó en ser procesado: 74seg
-----------------------------------------

Procesando el archivo: Pico_Atras_I1_20190410_080000.tar

Archivo 3 de un total de 10
El archivo tardó en ser procesado: 43seg
-----------------------------------------

Procesando el archivo: NoPico_Atras_I3_20190410_113001.tar

Archivo 4 de un total de 10
El archivo tardó en ser procesado: 74seg
-----------------------------------------

Procesando el archivo: Pico_Frente_I1_20190410_080000.tar

Archivo 5 de un total de 10
El archivo tardó en ser procesado: 44seg
-----------------------------------------

Procesando el archivo: Pico_Atras_I3_20190410_083001.tar

Archivo 6 de un total de 10
El archivo tardó en ser procesado: 71seg
----------------